In [35]:
# -*- coding: utf-8 -*-

import urllib
import urllib2
import re
from bs4 import BeautifulSoup
import sys
import os
import errno
import random
import string
from argparse import ArgumentParser
import time
from six.moves.urllib.parse import urlencode
import json 

def entryurl_googleimage(url, offset):
    splitkey1 = ''
    splitkey2 = ''
    if 'ijn=1'in url and 'start=100' in url:
        splitkey1 = 'ijn=1'
        splitkey2 = 'start=100'
    elif 'ijn=2' in url and 'start=200' in url:
        splitkey1 = 'ijn=2'
        splitkey2 = 'start=200'
    elif 'ijn=3' in url and 'start=300' in url:
        splitkey1 = 'ijn=3'
        splitkey2 = 'start=300'
    else:
        print 'ijn error'

    url = url.split(splitkey1)
    url = url = '{}ijn={}{}'.format(url[0],offset//100,url[1])
    
    url = url.split(splitkey2)
    url = url = '{}start={}{}'.format(url[0],offset,url[1])
    
    lang = ''
    if 'hl=en-HK' in url:
        lang = 'en'
    elif 'hl=zh-HK' in url:
        lang = 'zh'
    else:
        lang = 'zh'
    
    return url, lang
    
def response_contents(url):
    headers = {'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.94 Safari/537.36'}
    request = urllib2.Request(url, headers = headers)
    try:
        response = urllib2.urlopen(request)
        content = response.read()
    except urllib2.HTTPError as err:
        content = ''
        if err.code == 404:
            print 'HTTP Error 404: Not Found'
        else:
            raise

    return content

def extract_infos_googleimage(content, lang):
    img_id_url_dict = dict()
    
    if lang == 'zh':
        content = BeautifulSoup(content, 'lxml')
        try:
            content = str(json.loads(content.text))
        except ValueError:
            print("Oops!  That was No JSON object could be decoded.")
            return
    
    image_divs = BeautifulSoup(content, 'lxml').find_all('div', class_='rg_meta')
    for div in image_divs:
        try:
            meta = json.loads(div.text)
        except ValueError:
            print("Oops!  That was No JSON object could be decoded.")
            continue
            
        if 'ou' in meta and 'id' in meta and 'ity' in meta and 'ow' in meta and 'oh' in meta:
            minlen = int(meta['ow']) if meta['ow'] < meta['oh'] else meta['oh'] 
            if minlen < 350:
                continue
                
            if meta['ity'] == 'jpg' or meta['ity'] == 'jpeg' or len(meta['ity'])==0:
                if meta['id'] not in img_id_url_dict:
                    meta['id'] = meta['id'][0:len(meta['id'])-1]
                    img_id_url_dict[meta['id']] = meta['ou']
            else:
                print meta['ou'], meta['ity']
                
    return img_id_url_dict

def save_infos(img_id_url_dict, idprefix, fname):
    fid = open(fname, 'w')

    for imgid in img_id_url_dict:
        imgurl = img_id_url_dict[imgid]
        try:
            fid.write(idprefix+ '_' + str(imgid) + '   ' + imgurl + '\n')
        except Exception,e:
            print Exception,":",e

    fid.close()

def _mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc: # Python >2.5 (except OSError, exc: for Python <2.5)
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise
def main(keyword, initurl):
    offset = 0
    max_num = 1000
    idprefix='googleimage'
    extension = 'txt'
    outdir = '/home/lpzhang/Desktop/crawler/GoogleImages'
    
    img_dir = os.path.join(outdir, '_'.join(keyword.split()))
    _mkdir_p(img_dir)
    assert os.path.exists(img_dir), 'img_dir not exists'

    outfile= os.path.join(img_dir, '{}_{}_{}.{}'.format(idprefix, '_'.join(keyword.split()), len(os.listdir(img_dir)), extension))
    print outfile

    image_id_url_dict = dict()
    for i in range(offset, offset + max_num, 100):
        print 'start from', i
        url, lang = entryurl_googleimage(initurl, i)
        pages =  response_contents(url)
        img_id_url_dict = extract_infos_googleimage(pages, lang)

        image_id_url_dict.update(img_id_url_dict)

    print 'save image_id_url_dict'
    save_infos(image_id_url_dict, idprefix, outfile)

###############
# keyword = 'can opener'
urllist = list()

# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')
# urllist.append('')


for initurl in urllist:
    main(keyword, initurl)
    
print '########### done ##############'



/home/lpzhang/Desktop/crawler/GoogleImages/can_opener/googleimage_can_opener_8.txt
start from 0
https://cdn.shopify.com/s/files/1/1134/5960/products/Screen_Shot_2013_08_12_at_4.12.02_PM__04295.1378752207.1280.1280.png?v=1454076976 png
http://2.bp.blogspot.com/-oKDvfiPSHgg/T66QgXMItJI/AAAAAAAAAfA/dEF4426KTQE/s1600/Opened+Tin+Cans.png png
start from 100
http://3.bp.blogspot.com/-HoumytpoxDE/U0RajE6lfuI/AAAAAAAAGPY/XyDa_4LCGwM/s1600/can7.png png
start from 200
http://images.techhive.com/images/article/2014/02/canopener-iphone-player-100244671-large.png png
start from 300
http://upload.arabsbook.com/imgcache/5830.imgcache.gif gif
http://2.bp.blogspot.com/-Lkh-C-CNTs4/UD57_ZafAmI/AAAAAAAAIZA/ro1RidYOvYo/s1600/Screen+Shot+2012-08-29+at+20.02.36.png png
start from 400
start from 500
http://i303.photobucket.com/albums/nn157/Frostpebble/anime/hsm_geass.png png
http://4.bp.blogspot.com/-W9lrb23uhPM/VLES08aDn3I/AAAAAAAADIk/YqJ1tyMHxK4/s1600/In%2Bthe%2BCloset.png png
start from 600
http://www.buro